# Simulación de robots de limpieza — Actividad M.1
Este `.ipynb` contiene las secciones necesarias para recrear un simulador de limpieza basado en los siguientes elementos:

**Inputs:**
- Habitación de MxN espacios.
- Número de agentes.
- Porcentaje de celdas inicialmente sucias.
- Tiempo máximo de ejecución.

**Se simula lo siguiente:**
- Inicializa las celdas sucias (_ubicaciones aleatorias_).
- Todos los agentes empiezan en la celda [1,1].
- En cada paso de tiempo:
- Si la celda está sucia, entonces aspira.
- Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda).
- Se ejecuta el tiempo máximo establecido.

Todo lo anterior resulta en una animación de lo simulado, y se generan las siguientes **salidas**:
- Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
- Porcentaje de celdas limpias después del termino de la simulación.
- Número de movimientos realizados por todos los agentes.

Para esta simulación tomé un movimiento/frame como un segundo en tiempo real.
(1 frame = 1 seg).

**Notas:**
Se recomineda ejecutar las celdas en orden en un entorno Jupyter (con acceso a instalación pip), de preferencia en VSCode. En Colab va bien, pero, posterior a la ejecución de la primera sección, se debe de refrescar la página para reiniciar el entorno y poder usar la libreria de mesa.

In [7]:
# Instalar mesa si no está disponible e importar librerías
import sys, subprocess
try:
    import mesa
except Exception:
    print('Instalando mesa...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'mesa', 'ipywidgets'])
finally:
    import random, math, time
    from IPython.display import HTML, display, clear_output
    import matplotlib.pyplot as plt
    import matplotlib.animation as animation
    import numpy as np
    import mesa
    from mesa import Agent, Model
    from mesa.time import RandomActivation
    from mesa.space import MultiGrid
    import ipywidgets as widgets
print('Importación lista. (mesa v' + mesa.__version__ + ')')

Importación lista. (mesa v2.4.0)


In [2]:
# Definición del agente y del modelo (con _initial_dirty guardado)
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid

class VacuumAgent(Agent):
    """Agente aspiradora simple."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.moves = 0
        self.sucks = 0

    def step(self):
        x, y = self.pos
        if (x, y) in self.model.dirty_cells:
            # Aspirar
            self.model.dirty_cells.remove((x, y))
            self.sucks += 1
        else:
            neighbors = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            if neighbors:
                target = self.random.choice(neighbors)
                old_pos = self.pos
                self.model.grid.move_agent(self, target)
                if self.pos != old_pos:
                    self.moves += 1
                    self.model.total_moves += 1

class CleaningModel(Model):
    def __init__(self, width=10, height=10, n_agents=3, dirty_pct=0.3, max_time=200, seed=None):
        super().__init__()
        if seed is not None:
            self.seed = seed
            self.random.seed(seed)
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        self.width = width
        self.height = height
        self.n_agents = n_agents
        self.max_time = int(max_time)

        # inicializar celdas sucias
        total_cells = width * height
        n_dirty = int(total_cells * dirty_pct)
        all_coords = [(x, y) for x in range(width) for y in range(height)]
        start = (0, 0)  # interpretamos [1,1] del enunciado como (0,0) en 0-based
        possible_dirty = [c for c in all_coords if c != start]
        self.dirty_cells = set(self.random.sample(possible_dirty, min(n_dirty, len(possible_dirty))))
        # Guardar snapshot inicial (necesario para reconstrucción)
        self._initial_dirty = set(self.dirty_cells)

        # crear agentes en start
        for i in range(n_agents):
            a = VacuumAgent(i, self)
            self.schedule.add(a)
            self.grid.place_agent(a, start)

        # métricas y estados
        self.current_time = 0
        self.total_moves = 0
        self.time_to_clean_all = None
        self.history_agent_positions = []
        self.history_dirty_counts = []
        self.history_percent_clean = []

        # registrar estado inicial en history (t=0)
        init_positions = [agent.pos for agent in self.schedule.agents]
        self.history_agent_positions.append(list(init_positions))
        self.history_dirty_counts.append(len(self.dirty_cells))
        self.history_percent_clean.append(1.0 - len(self.dirty_cells)/(self.width*self.height))

    def step(self):
        self.schedule.step()
        self.current_time += 1
        positions = [agent.pos for agent in self.schedule.agents]
        self.history_agent_positions.append(list(positions))
        dirty_count = len(self.dirty_cells)
        self.history_dirty_counts.append(dirty_count)
        percent_clean = 1.0 - dirty_count / (self.width * self.height)
        self.history_percent_clean.append(percent_clean)
        if dirty_count == 0 and self.time_to_clean_all is None:
            self.time_to_clean_all = self.current_time

    def run_model(self):
        while self.current_time < self.max_time:
            self.step()
            if len(self.dirty_cells) == 0:
                break
        if self.time_to_clean_all is None:
            self.time_to_clean_all = self.current_time


In [ ]:
# run_and_get: ejecutar modelo y devolver métricas
import matplotlib.pyplot as plt
from IPython.display import HTML

def run_and_get(model_kwargs):
    m = CleaningModel(**model_kwargs)
    m.run_model()
    # construir snapshots (paso de tiempo en simulación) 
    # para animación (se hace después de ejecutar)
    build_snapshots(m)
    time_taken = m.time_to_clean_all
    percent_clean_final = 1.0 - len(m.dirty_cells)/(m.width*m.height)
    total_moves = m.total_moves
    return {'model': m, 'time_taken': time_taken, 'percent_clean_final': percent_clean_final, 'total_moves': total_moves}


In [ ]:
# Construcción de Snapshots
import numpy as np

def build_snapshots(runned_model):
    """Construye _dirty_snapshots y _moves_snapshots para la animación.
    Debe llamarse después de que el modelo haya corrido y tenga history_agent_positions."""
    width, height = runned_model.width, runned_model.height
    positions_history = runned_model.history_agent_positions
    T = len(positions_history)
    dirty = set(getattr(runned_model, '_initial_dirty', set(runned_model.dirty_cells)))
    dirty_snapshots = [set(dirty)]
    moves_snapshots = [0]
    total_moves = 0

    for t in range(1, T):
        prev_positions = positions_history[t-1]
        cur_positions = positions_history[t]
        # aspiraciones en 'prev_positions' (posiciones anteriores)
        for pos in prev_positions:
            if pos in dirty:
                dirty.discard(pos)
        # movimientos (comparando posiciones)
        moves_this_step = 0
        for p_prev, p_cur in zip(prev_positions, cur_positions):
            if p_prev != p_cur:
                moves_this_step += 1
        total_moves += moves_this_step
        dirty_snapshots.append(set(dirty))
        moves_snapshots.append(total_moves)

    runned_model._dirty_snapshots = dirty_snapshots
    runned_model._moves_snapshots = moves_snapshots
    return runned_model


In [ ]:
# Animación usando matplot porque el localhost 
# me tiraba errores de actualizacion
import matplotlib.animation as animation
from matplotlib.patches import Rectangle

def animate_simulation(model, interval=300):
    if not hasattr(model, '_dirty_snapshots') or not hasattr(model, '_moves_snapshots'):
        build_snapshots(model)

    width, height = model.width, model.height
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_xlim(-0.5, width-0.5)
    ax.set_ylim(-0.5, height-0.5)
    ax.set_xticks(range(width))
    ax.set_yticks(range(height))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_aspect('equal')
    plt.gca().invert_yaxis()

    cell_patches = {}
    for x in range(width):
        for y in range(height):
            rect = Rectangle((x-0.5, y-0.5), 1, 1, fill=True, edgecolor='black', linewidth=0.3)
            ax.add_patch(rect)
            cell_patches[(x,y)] = rect

    scat = ax.scatter([], [], s=200, marker='o')
    time_text = ax.text(0.02, 1.02, '', transform=ax.transAxes)
    cleaned_text = ax.text(0.02, 0.98, '', transform=ax.transAxes)
    moves_text = ax.text(0.02, 0.94, '', transform=ax.transAxes)

    def init():
        scat.set_offsets(np.empty((0,2)))
        time_text.set_text('')
        cleaned_text.set_text('')
        moves_text.set_text('')
        dirty_snapshot0 = model._dirty_snapshots[0] if len(model._dirty_snapshots) > 0 else set()
        for (x,y), rect in cell_patches.items():
            rect.set_facecolor('gray' if (x,y) in dirty_snapshot0 else 'white')
        return list(cell_patches.values()) + [scat, time_text, cleaned_text, moves_text]

    def update(frame):
        dirty_snapshot = model._dirty_snapshots[frame]
        for (x,y), rect in cell_patches.items():
            rect.set_facecolor('gray' if (x,y) in dirty_snapshot else 'white')

        positions = model.history_agent_positions[frame]
        if len(positions) == 0:
            coords = np.empty((0,2))
        else:
            coords = np.array([(x, y) for (x, y) in positions])
        scat.set_offsets(coords)

        t = frame
        time_text.set_text(f'Tiempo: {t}')
        cleaned = 1.0 - len(dirty_snapshot)/(model.width*model.height)
        cleaned_text.set_text(f'% limpio: {cleaned*100:.1f}%')
        moves_until = model._moves_snapshots[frame]
        moves_text.set_text(f'Movimientos acumulados: {moves_until}')
        return list(cell_patches.values()) + [scat, time_text, cleaned_text, moves_text]

    ani = animation.FuncAnimation(fig, update, frames=len(model.history_agent_positions), init_func=init,
                                  interval=interval, blit=False)
    plt.close(fig)
    return ani


In [ ]:
# Interfaz con ipywidgets: 
# ejecutar simulación, mostrar interfaz, animación y métricas
from IPython.display import display, clear_output

width_w = widgets.IntSlider(value=10, min=3, max=60, step=1, description='Ancho (M)')
height_w = widgets.IntSlider(value=10, min=3, max=60, step=1, description='Alto (N)')
agents_w = widgets.IntSlider(value=3, min=1, max=20, step=1, description='Agentes')
dirty_w = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.01, description='% sucio')
max_t_w = widgets.IntSlider(value=200, min=10, max=5000, step=10, description='Tiempo max')
run_btn = widgets.Button(description='Ejecutar simulación', button_style='success')
output = widgets.Output()

ui = widgets.VBox([widgets.HBox([width_w, height_w, agents_w]), widgets.HBox([dirty_w, max_t_w]), run_btn, output])


def on_run_clicked(b):
    with output:
        clear_output(wait=True)
        params = {
            'width': int(width_w.value),
            'height': int(height_w.value),
            'n_agents': int(agents_w.value),
            'dirty_pct': float(dirty_w.value),
            'max_time': int(max_t_w.value),
            'seed': None
        }
        print('Iniciando simulación con parámetros:', params)
        res = run_and_get(params)
        model = res['model']
        # mostrar métricas
        print('\n--- Resultados ---')
        print(f'Tiempo necesario (o tiempo alcanzado): {res["time_taken"]} Segundos (1 paso = 1 segundo)')
        print(f'Porcentaje de celdas limpias al final: {res["percent_clean_final"]*100:.2f}%')
        print(f'Número de movimientos realizados por todos los agentes: {res["total_moves"]}')

        print('\nGenerando animación...')
        ani = animate_simulation(model, interval=200)
        display(HTML(ani.to_jshtml()))

run_btn.on_click(on_run_clicked)

print('Interfaz lista. Ajusta parámetros y haz click en "Ejecutar simulación".')
ui


Interfaz lista. Ajusta parámetros y haz click en "Ejecutar simulación".


---

**Notas:**
- Este notebook guarda `_initial_dirty` en el constructor del modelo para reconstruir correctamente los snapshots y la animación.
- La animación usa `np.empty((0,2))` cuando no hay agentes visibles para evitar errores de `set_offsets([])`.